# KA Sandbox

### 11.1 Setup llm
Goal: Successful requests and responses to and from gemini

**Progress:**

- [x] Find a free version of api for testing
    - [x] Know what kind of access we need from TRK
- [x] Send request to gemini
- [x] Get response from gemini

### 11.3 Modify llm
Goal: Successful requests and responses with extended context to and from gemini

**Progress:**
- [x] Send context request to gemini
- [x] Get response using context from gemini
- [x] Try out stream version of gemini

### Consider two options for combining gemini and lovdata
#### 1. Retrive relevant laws from lovdata, then give as context to gemini

#### 2. Define a lovdata tool and give it to gemini to use when gemini deems it necessary

In [31]:
# General setup
import os
from google import genai
from dotenv import load_dotenv

load_dotenv(dotenv_path="../backend/.env")  # take environment variables from .env.

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [33]:
# Simple test of responses

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

AI learns patterns from data to make predictions or decisions.


In [25]:
from google.genai import types
from pydantic import BaseModel

class Dasq(BaseModel):
    recipe_name: str
    ingredients: list[str]

response = client.models.generate_content(
  model='gemini-2.5-flash',
  contents='List a few popular cookie recipes, and include the amounts of ingredients.',
  config=types.GenerateContentConfig(
      response_mime_type= 'application/json',
      response_schema= list[Dasq],
      seed=42,
  ),
)

In [26]:
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Dasq] = response.parsed

[
  {
    "recipe_name": "Classic Chocolate Chip Cookies",
    "ingredients": [
      "2 1/4 cups all-purpose flour",
      "1 teaspoon baking soda",
      "1 teaspoon salt",
      "1 cup (2 sticks) unsalted butter, softened",
      "3/4 cup granulated sugar",
      "3/4 cup packed brown sugar",
      "1 teaspoon vanilla extract",
      "2 large eggs",
      "2 cups (12 ounces) semi-sweet chocolate chips"
    ]
  },
  {
    "recipe_name": "Simple Sugar Cookies",
    "ingredients": [
      "3 cups all-purpose flour",
      "1 teaspoon baking powder",
      "1/2 teaspoon salt",
      "1 cup (2 sticks) unsalted butter, softened",
      "1 1/2 cups granulated sugar",
      "1 large egg",
      "1 teaspoon vanilla extract"
    ]
  },
  {
    "recipe_name": "Chewy Oatmeal Raisin Cookies",
    "ingredients": [
      "1 1/2 cups all-purpose flour",
      "1 teaspoon baking soda",
      "1 teaspoon ground cinnamon",
      "1/2 teaspoon salt",
      "1 cup (2 sticks) unsalted butter, softened",


In [27]:
article_6 = """Processing shall be lawful only if and to the extent that at least one of the following applies:
the data subject has given consent to the processing of his or her personal data for one or more specific purposes;
processing is necessary for the performance of a contract to which the data subject is party or in order to take steps at the request of the data subject prior to entering into a contract;
processing is necessary for compliance with a legal obligation to which the controller is subject;
processing is necessary in order to protect the vital interests of the data subject or of another natural person;
processing is necessary for the performance of a task carried out in the public interest or in the exercise of official authority vested in the controller;
processing is necessary for the purposes of the legitimate interests pursued by the controller or by a third party, except where such interests are overridden by the interests or fundamental rights and freedoms of the data subject which require protection of personal data, in particular where the data subject is a child.
Point (f) of the first subparagraph shall not apply to processing carried out by public authorities in the performance of their tasks.

Member States may maintain or introduce more specific provisions to adapt the application of the rules of this Regulation with regard to processing for compliance with points (c) and (e) of paragraph 1 by determining more precisely specific requirements for the processing and other measures to ensure lawful and fair processing including for other specific processing situations as provided for in Chapter IX.
1The basis for the processing referred to in point (c) and (e) of paragraph 1 shall be laid down by:
Union law; or
Member State law to which the controller is subject.
2The purpose of the processing shall be determined in that legal basis or, as regards the processing referred to in point (e) of paragraph 1, shall be necessary for the performance of a task carried out in the public interest or in the exercise of official authority vested in the controller. 3That legal basis may contain specific provisions to adapt the application of rules of this Regulation, inter alia: the general conditions governing the lawfulness of processing by the controller; the types of data which are subject to the processing; the data subjects concerned; the entities to, and the purposes for which, the personal data may be disclosed; the purpose limitation; storage periods; and processing operations and processing procedures, including measures to ensure lawful and fair processing such as those for other specific processing situations as provided for in Chapter IX. 4The Union or the Member State law shall meet an objective of public interest and be proportionate to the legitimate aim pursued.

Where the processing for a purpose other than that for which the personal data have been collected is not based on the data subject’s consent or on a Union or Member State law which constitutes a necessary and proportionate measure in a democratic society to safeguard the objectives referred to in Article 23(1), the controller shall, in order to ascertain whether processing for another purpose is compatible with the purpose for which the personal data are initially collected, take into account, inter alia:
any link between the purposes for which the personal data have been collected and the purposes of the intended further processing;
the context in which the personal data have been collected, in particular regarding the relationship between data subjects and the controller;
the nature of the personal data, in particular whether special categories of personal data are processed, pursuant to Article 9, or whether personal data related to criminal convictions and offences are processed, pursuant to Article 10;
the possible consequences of the intended further processing for data subjects;
the existence of appropriate safeguards, which may include encryption or pseudonymisation."""

In [28]:
context_text = article_6
user_question = "Can I use personal data collected for a contract to also send marketing emails?"

prompt = f"""
You are a legal assistant. Use the following law excerpts to answer the question:

Laws:
{context_text}

Question: {user_question}

Answer with citations from the provided laws only.
"""

In [41]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
      system_instruction='You are a legal assistant. Use the provided laws to answer the question in up to 5 short sentences. If the laws do not provide an answer, respond with "I do not know".',
      response_mime_type= 'text/plain',
      seed=42,
  ),
)

print(response.text)

Yes, if the data subject has given consent to the processing of their personal data for marketing purposes. (Processing shall be lawful only if... the data subject has given consent...)

Alternatively, processing for a purpose other than that for which the data was collected is permissible if based on a Union or Member State law. (Where the processing for a purpose other than that for which the personal data have been collected is not based on the data subject’s consent or on a Union or Member State law...)

If neither of these applies, the controller must assess whether processing for marketing is compatible with the original purpose of contract performance. (the controller shall, in order to ascertain whether processing for another purpose is compatible with the purpose for which the personal data are initially collected, take into account, inter alia:...)

This assessment involves considering factors such as any link between the purposes, the context of data collection, the nature o

In [47]:
response_stream = client.models.generate_content_stream(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction='You are a legal assistant. Use the provided laws to answer the question in up to 5 sentences. If the laws do not provide an answer, respond with "I do not know".',
        response_mime_type= 'text/plain',
        seed=42,
    ),
)

full_response = ""
for chunk in response_stream:
    print(chunk.text)
    print("HI")
    full_response += chunk.text

print(full_response)

Using personal data collected for a contract to send marketing emails constitutes processing for a purpose other than that for which the personal data was initially collected. Such further processing is permissible only if it is based on the data subject’s consent or on a Union or Member
HI
 State law. If neither of these conditions applies, the controller must ascertain whether processing for the marketing purpose is compatible with the purpose for which the personal data was initially collected. To do this, the controller must take into account factors such as any
HI
 link between the purposes, the context of data collection, the nature of the personal data, and the possible consequences for data subjects. The law does not provide a direct "yes" or "no" answer, but rather a framework for evaluating
HI
 the lawfulness and compatibility of such further processing.
HI
Using personal data collected for a contract to send marketing emails constitutes processing for a purpose other than th

In [ ]:
full_response = ""
for chunk in response_stream:
    print(chunk.text)
    print("HI")
    full_response += chunk.text

print(full_response)